## Imports

In [7]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
from tqdm import tqdm
import math
from scipy.stats import wasserstein_distance

## Declares

In [42]:
# MAC OSX
#WD = "/Users/alex/Documents/BUSTED_ModelTest"

WD = os.path.join("E:\\", "BUSTED_ModelTest")

# Dataset tag (User defined) ---
tag = "14-datasets"

# Additional declares
#BUSTEDS_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS")
#BUSTEDS_MH_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS-MH")
RESULTS_DIR = os.path.join(WD, "analysis")

# Create tables folder
OUTPUT_CSV = os.path.join(WD, "tables", "Table_" + tag + "_results_adjusted.csv")

ER_Threshold = 5

In [43]:
def read_json(filename):
    #print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        print("# -- Error -- file is empty:", filename)
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method

#define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
cv = lambda x: np.std(x) / np.mean(x)

pctchg = lambda a, b: (a / b) * 100

In [44]:
def process(FILES, fileending, method):
    df_dict = {}
    for item in tqdm(FILES):
        basename = ""
        if fileending in os.path.basename(item):
            basename = os.path.basename(item).replace(fileending, "")
        #end if
        
        for fext in [".phy", ".fasta", ".nex", "-align-dna.fas", "-Aligned-DNA.fas"]:
            basename = basename.replace(fext, "")
        #end for
        
        json_data = read_json(item) # Read json

        if json_data == []:
            continue # Empty file
        #end if
        df_dict[basename] = {"Method": method}
        df_dict[basename].update({"Sequences": json_data["input"]["number of sequences"]})
        df_dict[basename].update({"Codons": json_data["input"]["number of sites"]})
        df_dict[basename].update({"LRT p-value": json_data["test results"]["p-value"]})

        # cAIC
        df_dict[basename].update({"cAIC": json_data["fits"]["Unconstrained model"]["AIC-c"]})
        
        # Omegas
        A = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
        B = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
        C = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
        df_dict[basename].update({"CV(omega)": cv([A, B, C])})

        try:
            D = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
            E = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
            F = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
            df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
        except:
            pass
        #end try
        
        #Omegas and proportions
        data = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
        w1 = data["0"]["omega"]
        p1 = data["0"]["proportion"]
        w2 = data["1"]["omega"]
        p2 = data["1"]["proportion"]
        w3 = data["2"]["omega"]
        p3 = data["2"]["proportion"]
        df_dict[basename].update({"w1": w1, "p1": p1})
        df_dict[basename].update({"w2": w2, "p2": p2})
        df_dict[basename].update({"w3": w3, "p3": p3})

        # SRV rates and proportions
        if method == "BUSTEDS-MH" or method == "BUSTEDS":
            data = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
            s1 = data["0"]["rate"]
            s_p1 = data["0"]["proportion"]
            s2 = data["1"]["rate"]
            s_p2 = data["1"]["proportion"]
            s3 = data["2"]["rate"]
            s_p3 = data["2"]["proportion"]
            df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
            df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
            df_dict[basename].update({"SRV3": s3, "SRV_p3": s_p3})
        
        
        # DH rate, TH rate, TH_SI rate
        if method == "BUSTEDS-MH" or method == "BUSTED-MH":
            df_dict[basename].update({"DH_Rate": float(json_data["fits"]["Unconstrained model"]
                                  ["rate at which 2 nucleotides are changed instantly within a single codon"])})
            df_dict[basename].update({"TH_Rate": float(json_data["fits"]["Unconstrained model"]
                                  ["rate at which 3 nucleotides are changed instantly within a single codon"])})
            df_dict[basename].update({"TH_Rate_SI": float(json_data["fits"]["Unconstrained model"]
                                  ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"])})

        
        # ER Sites -- Contrained
        ER_SITES = []
        ER_df_dict = {}
        if "constrained" in json_data["Evidence Ratios"].keys():
            for site, val in enumerate(json_data["Evidence Ratios"]["constrained"][0]):
                if val > ER_Threshold:
                    ER_SITES.append(str(site + 1))
                    ER_df_dict[site + 1] = {"ER": val}
                #end if
            #end for
            df_dict[basename].update({"NUM_ER_SITES_CONSTRAINED":  len(ER_df_dict.keys())})
            x = ER_df_dict.keys()
            x = [str(x) for x in x]
            df_dict[basename].update({"ER_SITES_CONSTRAINED":  "|".join(x)})
        #end if
        
        # ER Sites -- Optimized null
        ER_SITES = []
        ER_df_dict = {}
        ON_tag = "optimized null"
        if ON_tag in json_data["Evidence Ratios"].keys():
            for site, val in enumerate(json_data["Evidence Ratios"][ON_tag][0]):
                if val > ER_Threshold:
                    ER_SITES.append(str(site + 1))
                    ER_df_dict[site + 1] = {"ER": val}
                #end if
            #end for
            df_dict[basename].update({"NUM_ER_SITES_OPTIMIZED_NULL":  len(ER_df_dict.keys())})
            x = ER_df_dict.keys()
            x = [str(x) for x in x]
            df_dict[basename].update({"ER_SITES_OPTIMIZED_NULL":  "|".join(x)})
        #end if
        
        
    # End for
    df = pd.DataFrame.from_dict(df_dict, orient="index")
    df = df.reset_index()
    df.index += 1
    df.rename(columns={'index': 'Gene'}, inplace = True)
    return df
#end method

## Look over results

In [45]:
BUSTEDS_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTEDS.json")]
BUSTEDS_MH_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTEDS-MH.json")]
BUSTED_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTED.json")]
BUSTED_MH_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTED-MH.json")]

print("# Number of BUSTEDS results:", len(BUSTEDS_RESULTS))
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_RESULTS))
print("# Number of BUSTED results:", len(BUSTED_RESULTS))
print("# Number of BUSTED-MH results:", len(BUSTED_MH_RESULTS))

#print("# Number of SLAC results:", len(SLAC_DIR_FILES))

# Number of BUSTEDS results: 14
# Number of BUSTEDS-MH results: 14
# Number of BUSTED results: 14
# Number of BUSTED-MH results: 14


In [46]:
print("# Processing BUSTED[S] files")
df_BUSTEDS = process(BUSTEDS_RESULTS , ".BUSTEDS.json", "BUSTEDS")

print("# Processing BUSTED[S]-MH files")
df_BUSTEDS_MH = process(BUSTEDS_MH_RESULTS , ".BUSTEDS-MH.json", "BUSTEDS-MH")

print("# Processing BUSTED files")
df_BUSTED = process(BUSTED_RESULTS , ".BUSTED.json", "BUSTED")

print("# Processing BUSTED-MH files")
df_BUSTED_MH = process(BUSTED_MH_RESULTS, ".BUSTED-MH.json", "BUSTED-MH")


# Processing BUSTED[S] files


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 333.42it/s]


# Processing BUSTED[S]-MH files


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 368.45it/s]


# Processing BUSTED files


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 500.03it/s]


# Processing BUSTED-MH files


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 437.56it/s]


In [47]:
df_BUSTEDS

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),CV(alpha),w1,p1,...,SRV1,SRV_p1,SRV2,SRV_p2,SRV3,SRV_p3,NUM_ER_SITES_CONSTRAINED,ER_SITES_CONSTRAINED,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL
1,adh,BUSTEDS,23,254,1.815529e-03,9354.009509,1.383019,0.574009,0.020131,0.173311,...,0.605146,0.268544,0.743412,0.506222,2.047467,0.225235,18.0,4|6|35|39|46|49|68|69|133|134|163|165|166|170|...,1.0,170
2,bglobin,BUSTEDS,17,144,8.467278e-06,7415.047518,1.304447,1.087483,0.035399,0.828310,...,0.339402,0.386773,1.056381,0.579178,7.544826,0.034049,10.0,10|11|42|48|50|54|74|110|116|124,5.0,10|42|48|50|110
3,camelid,BUSTEDS,212,96,0.000000e+00,33694.129302,1.334016,0.822533,0.009197,0.312471,...,0.209278,0.310758,0.814475,0.465327,2.482944,0.223915,33.0,1|10|14|23|24|25|29|30|32|33|35|39|40|41|49|50...,28.0,5|10|23|24|25|26|28|30|32|33|35|40|50|51|52|54...
4,COXI,BUSTEDS,21,510,5.000000e-01,24288.101212,1.381566,1.330024,0.000977,0.673833,...,0.033245,0.018959,0.575680,0.950062,14.604620,0.030979,NaN,NaN,NaN,NaN
5,ENCenv,BUSTEDS,23,500,5.000000e-01,13699.103102,1.254676,0.893186,0.031310,0.160025,...,0.403572,0.305010,1.134033,0.669239,4.581135,0.025751,NaN,NaN,NaN,NaN
6,flavNS5,BUSTEDS,18,342,4.826393e-01,18530.532597,1.352523,1.215819,0.008133,0.796769,...,0.120672,0.187685,0.524341,0.714772,6.177431,0.097543,0.0,,0.0,
7,HepatitisD,BUSTEDS,33,196,1.124733e-08,10424.144462,1.332800,0.974870,0.000000,0.512881,...,0.035100,0.209307,0.758645,0.596156,2.777791,0.194537,23.0,6|9|13|17|24|28|31|35|38|75|85|90|117|122|140|...,8.0,13|24|35|90|117|140|145|183
8,HIVvif,BUSTEDS,29,192,2.383380e-02,6913.790452,1.411915,0.827416,0.716583,0.990546,...,0.295969,0.545601,1.158556,0.325050,3.571189,0.129349,1.0,6,1.0,6
9,HIV_RT,BUSTEDS,476,335,1.535105e-12,52026.769254,1.381185,0.833936,0.122886,0.962661,...,0.402784,0.515996,1.209478,0.412551,4.103288,0.071453,17.0,48|64|69|75|104|122|138|151|162|163|181|188|21...,10.0,48|64|135|151|163|178|188|200|215|228
10,InfluenzaA,BUSTEDS,349,329,1.063992e-01,23242.618200,0.837749,0.870636,0.097600,0.771979,...,0.426581,0.656498,1.692692,0.310003,5.827420,0.033499,31.0,50|80|121|128|133|135|137|138|142|145|156|157|...,0.0,


In [48]:
df_BUSTEDS_MH

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),CV(alpha),w1,p1,...,SRV_p2,SRV3,SRV_p3,DH_Rate,TH_Rate,TH_Rate_SI,NUM_ER_SITES_CONSTRAINED,ER_SITES_CONSTRAINED,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL
1,adh,BUSTEDS-MH,23,254,0.019489,9359.111669,1.383861,0.552316,1.615062e-02,0.107078,...,0.251789,2.808413,0.074966,0.000000,0.033096,3.364916,15.0,6|35|39|49|69|133|134|163|165|166|170|197|216|...,0.0,
2,bglobin,BUSTEDS-MH,17,144,0.002283,7384.827435,1.314964,1.062452,4.399043e-02,0.828239,...,0.597619,6.942281,0.034317,0.000000,0.000000,259.102523,11.0,10|11|42|48|50|54|74|110|116|124|133,1.0,110
3,camelid,BUSTEDS-MH,212,96,0.006905,33668.701415,0.922319,0.804136,4.813634e-01,0.776627,...,0.474061,2.454190,0.218972,0.173423,0.016813,0.000000,26.0,1|11|14|23|25|29|31|32|33|40|50|51|52|53|54|55...,0.0,
4,COXI,BUSTEDS-MH,21,510,0.500000,24295.058286,0.701211,1.306716,5.584007e-03,0.989112,...,0.945608,12.158473,0.034107,0.000000,0.000000,4.371442,NaN,NaN,NaN,NaN
5,ENCenv,BUSTEDS-MH,23,500,0.500000,13705.123029,1.282742,0.891511,4.902972e-02,0.937270,...,0.664743,4.587889,0.025533,0.012449,0.000000,0.000000,NaN,NaN,NaN,NaN
6,flavNS5,BUSTEDS-MH,18,342,0.500000,18488.578696,1.380767,1.079050,6.219587e-08,0.000000,...,0.662687,4.399485,0.122471,0.362455,0.907991,2.395013,NaN,NaN,NaN,NaN
7,HepatitisD,BUSTEDS-MH,33,196,0.492512,10423.693064,1.263528,0.920703,0.000000e+00,0.414562,...,0.565875,2.666951,0.194622,0.242721,0.000000,0.000000,0.0,,0.0,
8,HIVvif,BUSTEDS-MH,29,192,0.500000,6913.086831,0.252332,0.820836,5.849111e-01,0.119606,...,0.326605,3.507036,0.131647,0.001566,0.162595,0.000000,NaN,NaN,NaN,NaN
9,HIV_RT,BUSTEDS-MH,476,335,0.001235,52037.043272,1.407171,0.826657,0.000000e+00,0.018400,...,0.414642,3.987729,0.075316,0.039729,0.000000,0.000000,12.0,48|64|69|75|122|151|162|181|188|215|228|245,3.0,188|215|228
10,InfluenzaA,BUSTEDS-MH,349,329,0.500000,23230.195553,0.474401,0.877723,3.619313e-01,0.000000,...,0.319125,5.883673,0.032359,0.062803,0.014882,0.000000,NaN,NaN,NaN,NaN


In [49]:
df_BUSTED

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),w1,p1,w2,p2,w3,p3,NUM_ER_SITES_CONSTRAINED,ER_SITES_CONSTRAINED,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL
1,adh,BUSTED,23,254,2.624235e-04,9371.692413,1.085644,0.033896,0.968429,1.000000,0.009224,5.330541,0.022347,24.0,2|4|35|39|49|57|68|69|81|98|133|134|163|165|16...,2.0,163|170
2,bglobin,BUSTED,17,144,1.161271e-11,7452.740235,1.313216,0.044568,0.853287,1.000000,0.119840,20.657268,0.026872,18.0,3|7|10|11|18|42|48|49|50|54|67|74|85|110|114|1...,9.0,10|42|48|50|85|110|116|123|124
3,camelid,BUSTED,212,96,0.000000e+00,35140.302106,1.277873,0.496019,0.915425,1.000000,0.033814,21.733517,0.050761,41.0,1|11|14|23|24|25|27|28|29|30|31|32|33|35|40|44...,47.0,4|5|8|9|10|12|17|18|20|21|23|24|25|26|28|29|30...
4,COXI,BUSTED,21,510,5.000000e-01,24384.439201,0.839072,0.000000,0.956700,0.522091,0.043300,1.132240,0.000000,NaN,NaN,NaN,NaN
5,ENCenv,BUSTED,23,500,5.000000e-01,13745.467295,1.203131,0.025662,0.627805,0.099957,0.372195,1.123752,0.000000,NaN,NaN,NaN,NaN
6,flavNS5,BUSTED,18,342,4.220719e-01,18789.833822,1.390279,0.005805,0.949834,0.007204,0.005898,1.139867,0.044268,0.0,,0.0,
7,HepatitisD,BUSTED,33,196,0.000000e+00,10697.519123,1.338044,0.000000,0.581165,0.613151,0.389446,16.299388,0.029388,43.0,4|6|8|9|11|13|17|23|24|28|31|35|38|47|56|62|75...,16.0,6|9|13|23|24|28|31|35|90|117|122|140|145|150|1...
8,HIVvif,BUSTED,29,192,2.005439e-04,7101.856673,1.178539,0.330011,0.630743,0.422008,0.285168,6.013574,0.084090,32.0,6|19|22|31|33|36|37|39|41|47|48|51|63|66|91|92...,2.0,31|109
9,HIV_RT,BUSTED,476,335,0.000000e+00,53749.794833,1.350796,0.079227,0.935271,1.000000,0.060956,34.814021,0.003773,46.0,35|36|39|48|49|64|65|69|75|83|98|100|103|104|1...,16.0,35|48|64|75|104|122|151|162|163|174|181|188|20...
10,InfluenzaA,BUSTED,349,329,6.970205e-09,23863.727926,1.330142,0.347174,0.986079,1.000000,0.009692,32.560865,0.004229,25.0,15|53|121|133|135|137|138|145|156|157|159|172|...,5.0,133|135|157|159|213


In [50]:
df_BUSTED_MH

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),w1,p1,w2,p2,w3,p3,DH_Rate,TH_Rate,TH_Rate_SI,NUM_ER_SITES_CONSTRAINED,ER_SITES_CONSTRAINED,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL
1,adh,BUSTED-MH,23,254,2.107436e-02,9377.068424,1.394360,0.000000,0.040880,0.037392,0.932528,3.948384,0.026592,0.087721,0.000000,0.000000,24.0,2|4|35|39|49|57|68|69|81|98|133|134|163|165|16...,0.0,
2,bglobin,BUSTED-MH,17,144,3.809158e-03,7427.727504,1.385364,0.000000,0.053339,0.096493,0.897213,6.973918,0.049448,0.092113,0.000000,266.862999,23.0,3|7|8|10|11|14|42|48|49|50|54|67|74|85|107|110...,3.0,11|48|110
3,camelid,BUSTED-MH,212,96,3.885781e-15,35092.213800,1.261640,0.427856,0.209959,0.560847,0.735838,12.752791,0.054203,0.164634,0.000000,0.521829,37.0,1|11|14|23|24|25|27|28|29|30|31|32|33|35|40|44...,15.0,11|25|29|32|33|41|50|51|52|54|55|56|57|67|78
4,COXI,BUSTED-MH,21,510,5.000000e-01,24389.451070,0.814080,0.000000,0.955725,0.505647,0.044275,1.002329,0.000000,0.000000,0.000000,5.676599,NaN,NaN,NaN,NaN
5,ENCenv,BUSTED-MH,23,500,5.000000e-01,13749.208238,1.278712,0.017653,0.000000,0.051048,1.000000,1.002331,0.000000,0.086759,0.000000,0.000000,NaN,NaN,NaN,NaN
6,flavNS5,BUSTED-MH,18,342,5.000000e-01,18765.530277,1.349079,0.005386,0.482682,0.026407,0.479249,1.000000,0.038069,0.433452,0.015393,4.352182,NaN,NaN,NaN,NaN
7,HepatitisD,BUSTED-MH,33,196,1.849457e-03,10677.994529,1.354563,0.000000,0.059398,0.175972,0.879149,6.035729,0.061453,0.252173,0.000000,0.000000,43.0,4|6|8|9|11|13|16|17|23|24|28|38|47|48|56|62|75...,0.0,
8,HIVvif,BUSTED-MH,29,192,4.292228e-02,7100.266446,1.414214,0.000000,0.630209,0.000000,0.037092,2.389983,0.332699,0.000000,0.180808,0.000000,36.0,20|22|31|33|36|37|39|41|47|48|50|51|61|63|66|9...,0.0,
9,HIV_RT,BUSTED-MH,476,335,0.000000e+00,53757.215664,1.404597,0.000000,0.053308,0.133114,0.942023,29.197316,0.004669,0.000000,0.000000,0.000000,48.0,35|36|39|48|49|60|64|65|69|75|83|98|100|103|10...,12.0,48|64|75|122|151|162|174|181|188|215|228|245
10,InfluenzaA,BUSTED-MH,349,329,4.801551e-01,23861.491457,0.923508,0.000054,0.542831,0.393790,0.095792,1.135820,0.361377,0.067817,0.014964,0.000000,6.0,138|145|156|186|193|226,0.0,


## Concatenate tables


In [37]:
result = pd.concat([df_BUSTEDS, df_BUSTEDS_MH, df_BUSTED, df_BUSTED_MH])
result = result.fillna("")
result = result.sort_values(by=["Gene", "Method"], ascending=True)
result = result.reset_index(drop=True)
result.index += 1
result


,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),CV(alpha),w1,p1,...,SRV_p2,SRV3,SRV_p3,NUM_ER_SITES_CONSTRAINED,ER_SITES_CONSTRAINED,NUM_ER_SITES_OPTIMIZED_NULL,ER_SITES_OPTIMIZED_NULL,DH_Rate,TH_Rate,TH_Rate_SI
1,COXI,BUSTED,21,510,5.000000e-01,24384.439201,0.839072,,0.000000e+00,0.956700,...,,,,,,,,,,
2,COXI,BUSTED-MH,21,510,5.000000e-01,24389.451070,0.814080,,0.000000e+00,0.955725,...,,,,,,,,,,
3,COXI,BUSTEDS,21,510,5.000000e-01,24288.101212,1.381566,1.330024,9.774121e-04,0.673833,...,0.950062,14.60462,0.030979,,,,,,,
4,COXI,BUSTEDS-MH,21,510,5.000000e-01,24295.058286,0.701211,1.306716,5.584007e-03,0.989112,...,0.945608,12.158473,0.034107,,,,,0.0,0.0,4.371442
5,ENCenv,BUSTED,23,500,5.000000e-01,13745.467295,1.203131,,2.566163e-02,0.627805,...,,,,,,,,,,
6,ENCenv,BUSTED-MH,23,500,5.000000e-01,13749.208238,1.278712,,1.765263e-02,0.000000,...,,,,,,,,,,
7,ENCenv,BUSTEDS,23,500,5.000000e-01,13699.103102,1.254676,0.893186,3.131041e-02,0.160025,...,0.669239,4.581135,0.025751,,,,,,,
8,ENCenv,BUSTEDS-MH,23,500,5.000000e-01,13705.123029,1.282742,0.891511,4.902972e-02,0.937270,...,0.664743,4.587889,0.025533,,,,,0.012449,0.0,0.0
9,HIV_RT,BUSTED,476,335,0.000000e+00,53749.794833,1.350796,,7.922727e-02,0.935271,...,,,,46.0,35|36|39|48|49|64|65|69|75|83|98|100|103|104|1...,16.0,35|48|64|75|104|122|151|162|163|174|181|188|20...,,,
10,HIV_RT,BUSTED-MH,476,335,0.000000e+00,53757.215664,1.404597,,0.000000e+00,0.053308,...,,,,48.0,35|36|39|48|49|60|64|65|69|75|83|98|100|103|10...,12.0,48|64|75|122|151|162|174|181|188|215|228|245,,,


In [38]:
result.columns

Index(['Gene', 'Method', 'Sequences', 'Codons', 'LRT p-value', 'cAIC',
       'CV(omega)', 'CV(alpha)', 'w1', 'p1', 'w2', 'p2', 'w3', 'p3', 'SRV1',
       'SRV_p1', 'SRV2', 'SRV_p2', 'SRV3', 'SRV_p3',
       'NUM_ER_SITES_CONSTRAINED', 'ER_SITES_CONSTRAINED',
       'NUM_ER_SITES_OPTIMIZED_NULL', 'ER_SITES_OPTIMIZED_NULL', 'DH_Rate',
       'TH_Rate', 'TH_Rate_SI'],
      dtype='object')

## Save table

In [41]:
print("Saving results to:", OUTPUT_CSV)
result.to_csv(OUTPUT_CSV, index=False)

Saving results to: E:\BUSTED_ModelTest\tables\Table_14-datasets_results_adjusted.csv


## End of file